# 1. Import Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random

# 2. Write Function

In [2]:
def SVM_compute_E(W, x_train, b, y):
    E = np.dot(x_train, W) + b - y
    return E

In [3]:
def SVM_compute_eta(xi, xj, kf):
    eta = 2*kernel(xi, xj, kf) - kernel(xi, xi, kf) - kernel(xj, xj, kf)
    return eta

In [4]:
def SVM_compute_L(yi, yj, C, alpha_i, alpha_j):
    if yi == yj:
        L = max([0, alpha_i + alpha_j - C])
    elif yi != yj:
        L = max([0, alpha_j - alpha_i])
    return L

In [5]:
def SVM_compute_H(yi, yj, C, alpha_i, alpha_j):
    if yi == yj:
        H = min([C, alpha_i + alpha_j])
    elif yi != yj:
        H = min([C, C + alpha_j - alpha_i])
    return H

In [6]:
def SVM_compute_alpha_j(alpha_j, yj, Ei, Ej, eta, L, H):
    new_alpha_j = alpha_j - yj*(Ei - Ej)/eta
    if new_alpha_j < L:
        new_alpha_j = L
    elif new_alpha_j > H:
        new_alpha_j = H
    return new_alpha_j

In [7]:
def SVM_compute_b(b, Ei, Ej, new_alpha_i, old_alpha_i, new_alpha_j, old_alpha_j, yi, xi, yj, xj, C, kf):
    bi = b - Ei - (new_alpha_i - old_alpha_i)*yi*kernel(xi, xi, kf) - (new_alpha_j - old_alpha_j)*yj*kernel(xi, xj, kf)
    bj = b - Ej - (new_alpha_j - old_alpha_j)*yj*kernel(xj, xj, kf) - (new_alpha_i - old_alpha_i)*yi*kernel(xi, xj, kf)
    is_sv_i = (new_alpha_i > 0) and (new_alpha_i < C)
    is_sv_j = (new_alpha_j > 0) and (new_alpha_j < C)
    if is_sv_i and not is_sv_j:
        new_b = bi
    elif not is_sv_i and  is_sv_j:
        new_b = bj
    else:
        new_b = (bi + bj)/2
    return new_b

In [8]:
def kernel(xi, xj, kf):
    if kf[0] == 'Dot':
        return np.dot(xi, xj)
    elif kf[0] == 'Polynomial':
        p = kf[1]
        return (np.dot(xi, xj) + 1)**p
    elif kf[0] == 'RBF':
        gamma = kf[1]
        degree = np.sum((xi - xj)**2)
        return np.e**(-gamma*degree)

In [9]:
def SVM_fit(X_Train, Y_Train, C, kf, max_iteration = 1000):
    N = X_Train.shape[0]
    D = X_Train.shape[1]
    alpha = np.zeros(N)
    W = np.zeros(D)
    b = 0
    for e in range(max_iteration):
        two_dots_index = random.sample(range(N), 2)
        i = two_dots_index[0]
        xi = X_Train[i, :]
        yi = Y_Train[i, 0]
        j = two_dots_index[1]
        xj = X_Train[j, :]
        yj = Y_Train[j, 0]
        Ei = SVM_compute_E(W, xi, b, yi)
        Ej = SVM_compute_E(W, xj, b, yj)
        eta = SVM_compute_eta(xi, xj, kf)
        if eta >= 0:
            continue
        L = SVM_compute_L(yi, yj, C, alpha[i], alpha[j])
        H = SVM_compute_H(yi, yj, C, alpha[i], alpha[j])
        new_alpha_j = SVM_compute_alpha_j(alpha[j], yj, Ei, Ej, eta, L, H)
        new_alpha_i = alpha[i] + yi*yj*(alpha[j] - new_alpha_j)
        b = SVM_compute_b(b, Ei, Ej, new_alpha_i, alpha[i], new_alpha_j, alpha[j], yi, xi, yj, xj, C, kf)
        W = W + (new_alpha_i - alpha[i])*yi*xi + (new_alpha_j - alpha[j])*yj*xj
        alpha[i] = new_alpha_i
        alpha[j] = new_alpha_j
    return W, b, alpha

In [10]:
def SVM_predict(X_Test, W, b):
    Zhat_Test = np.dot(X_Test, W) + b
    Yhat_Test = np.sign(Zhat_Test)
    return Yhat_Test.reshape(-1, 1)

In [11]:
def min4norm(Data):
    _min = Data.min(axis=0)
    return _min.reshape(1, -1)

In [12]:
def max4norm(Data):
    _max = Data.max(axis=0)
    return _max.reshape(1, -1)

In [13]:
def minmaxNorm(Data, _min, _max):
    Data_Norm = (Data - _min)/(_max - _min)
    return Data_Norm

In [14]:
def de_minmaxNorm(Data_Norm, _min, _max):
    Data = Data_Norm*(_max - _min) + _min
    return Data

In [15]:
def find_error_classification(Y, Yhat):
    N = Y.shape[0]
    error = (100/N)*(Y != Yhat).sum()
    return error

# 3. Read Data & Prepare Data

In [16]:
Data = pd.read_excel('breast_cancer.xlsx')

In [17]:
Data

,radius (mean),texture (mean),perimeter (mean),area (mean),smoothness (mean),compactness (mean),concavity (mean),concave points (mean),symmetry (mean),fractal dimension (mean),...,texture (worst),perimeter (worst),area (worst),smoothness (worst),compactness (worst),concavity (worst),concave points (worst),symmetry (worst),fractal dimension (worst),class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [18]:
DataMatrix = Data.values

In [19]:
np.random.shuffle(DataMatrix)

In [20]:
DataMatrix

array([[2.116e+01, 2.304e+01, 1.372e+02, ..., 2.822e-01, 7.526e-02,
        0.000e+00],
       [1.005e+01, 1.753e+01, 6.441e+01, ..., 2.894e-01, 7.664e-02,
        1.000e+00],
       [1.720e+01, 2.452e+01, 1.142e+02, ..., 3.313e-01, 1.339e-01,
        0.000e+00],
       ...,
       [1.048e+01, 1.986e+01, 6.672e+01, ..., 2.883e-01, 7.748e-02,
        1.000e+00],
       [1.210e+01, 1.772e+01, 7.807e+01, ..., 3.049e-01, 7.081e-02,
        1.000e+00],
       [1.283e+01, 2.233e+01, 8.526e+01, ..., 3.407e-01, 1.243e-01,
        0.000e+00]])

In [21]:
N = DataMatrix.shape[0]

In [22]:
D = DataMatrix.shape[1] - 1

In [23]:
X = DataMatrix[:, :D]
Y = DataMatrix[:, D:D+1]

In [24]:
Y = np.where(Y == 1, Y, -1)

In [25]:
start_train = 0
end_train = -150
# end_valid = -50
# end_test = -1

In [26]:
X_Train = X[start_train:end_train, :]
Y_Train = Y[start_train:end_train, :]

# X_Valid = X[end_train:end_valid, :]
# Y_Valid = Y[end_train:end_valid, :]

X_Test = X[end_train:, :]
Y_Test = Y[end_train:, :]

In [27]:
_min = min4norm(X_Train)
_max = max4norm(X_Train)

X_Train_Norm = minmaxNorm(X_Train, _min, _max)
X_Test_Norm = minmaxNorm(X_Test, _min, _max)

# 4. Create Model

In [53]:
W, b, alpha = SVM_fit(X_Train_Norm, Y_Train, 1, ['RBF', 3])
# W, b, alpha = SVM_fit(X_Train_Norm, Y_Train, 1, ['Polynomial', 3])
# W, b, alpha = SVM_fit(X_Train_Norm, Y_Train, 1, ['Dot', 3])

In [54]:
W

array([-0.85440236, -1.41916965, -0.82437092, -0.80881595, -0.65230335,
        0.16354082, -0.88737521, -0.99263189,  0.0934404 ,  0.56123478,
       -0.91178177,  0.06559419, -0.55333817, -0.59754199, -0.67546091,
        0.66061284,  0.09148846,  0.01379144,  0.38298912,  0.29075144,
       -1.51497339, -1.39111586, -1.26940941, -1.31226033, -1.44212129,
       -0.07107682, -0.85511106, -1.09507007, -0.60492906, -0.21075288])

In [55]:
b

4.722214936851888

In [56]:
alpha

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.63728023e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.48822270e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.16518209e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 6.28997881e-01, 7.81506632e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.53701725e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.65527886e-01,
      

# 5. Making Prediction

In [57]:
Yhat_Test = SVM_predict(X_Test_Norm, W, b)

In [58]:
error_Test = find_error_classification(Y_Test, Yhat_Test)

In [59]:
error_Test

6.666666666666666

In [60]:
np.hstack([Yhat_Test, Y_Test])

array([[-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [-1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [-1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1.,